<a href="https://colab.research.google.com/github/rsin46/pegasus-demo/blob/master/Pegasus_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# MODEL: gigaword for gigaword
#        cnn_dailymail for CNN Dailymail
# 1597317576 - Batch of 8
# 1597369292 - No batch
MODELNAME = 'aeslc'
MODEL_DIR = '1597317576'
BATCH_SIZE = 8

In [3]:
!git clone http://github.com/rsin46/pegasus-demo.git
!git clone http://github.com/google-research/pegasus.git

Cloning into 'pegasus-demo'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 2), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
Cloning into 'pegasus'...
remote: Enumerating objects: 171, done.
remote: Total 171 (delta 0), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (171/171), 353.09 KiB | 1.30 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [4]:
!mkdir /content/pegasus-demo/model
!pip install sentencepiece
# !pip install -e pegasus
# !pip install tensorflow_transform

     |████████████████████████████████| 1.1MB 8.6MB/s 


In [5]:
if MODELNAME == 'gigaword':
  !cp '/content/gdrive/My Drive/Models/gigaword.tar.xz' '/content/pegasus-demo/model/gigaword.tar.xz'
  !tar -xvf '/content/pegasus-demo/model/gigaword.tar.xz' -C '/content/pegasus-demo/model/' 
elif MODELNAME == 'cnn_dailymail':
  !cp '/content/gdrive/My Drive/Models/cnn_dailymail.xz' '/content/pegasus-demo/model/cnn_dailymail.xz'
  !tar -xvf '/content/pegasus-demo/model/cnn_dailymail.xz' -C '/content/pegasus-demo/model/'
elif MODELNAME == 'aeslc':
  source = '/content/gdrive/My\ Drive/Models/{}'.format(MODEL_DIR)
  !cp -r $source '/content/pegasus-demo/model/'


In [6]:
%cd /content/pegasus-demo
# Initialising variables

TEXT = 'thermo.txt'
OUT_TEXT = '/content/gdrive/My Drive/Models/' + TEXT

def append(line):
    with open(OUT_TEXT, 'a', buffering=1) as redf:
        redf.write(line)

/content/pegasus-demo


In [7]:
import text_eval
import public_parsing_ops
import tensorflow as tf
import numpy as np
import datetime
import time

_SPM_VOCAB = 'ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece",
                                                 _SPM_VOCAB)

shapes = {
    'cnn_dailymail': (1024, 128),
    'gigaword':(128, 32),
    'aeslc':(512, 32)
}

model_dir = {
    'cnn_dailymail': 'model/1595037231/',
    'gigaword': 'model/1595095633/',
    'aeslc': 'model/{}'.format(MODEL_DIR)
}

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


imported = tf.saved_model.load(model_dir[MODELNAME], tags='serve')


text = open(TEXT, "r", encoding="utf-8").read()


shape,_ = shapes[MODELNAME]

input_ids_raw = encoder.encode(text)
input_ids_split = list(zip(*([iter(input_ids_raw)]*shape)))
batch_list = list(chunks(input_ids_split,BATCH_SIZE))

out_file = open(OUT_TEXT,"a")
current_time = datetime.datetime.now()
append(current_time.strftime("%d-%b-%Y (%H:%M:%S.%f)") + '\n')

for batch in batch_list:
  start = time.time()
  
  input_list = []

  for input_ids in batch:
    inputs = np.zeros(shape)
    idx = len(input_ids)
    if idx>shape: idx =shape

    inputs[:idx] = input_ids[:idx]

    example = tf.train.Example()

    example.features.feature["inputs"].int64_list.value.extend(inputs.astype(int))

    input_list.append(tf.constant([example.SerializeToString()]))
  
  # Pad if the last list doesn't fit into a batch
  if len(input_list) < BATCH_SIZE:
    for i in range(0,BATCH_SIZE - len(input_list):
      input_list.append(input_list[-1])

  output = imported.signatures['serving_default'](examples=tf.concat(input_list, axis=0))
  prediction = text_eval.ids2str(encoder, output['outputs'].numpy(), None)
  end = time.time()
  duration = end - start
  print("{}\nPREDICTION >> ".format(duration), prediction)
  append(f'{duration}s ... {prediction} \n')


11.835957288742065
PREDICTION >>  Second Law of Thermo Dynamics Book 1 Final Exam Gas Models.xls (no subject) Low-Temperature and Proposals (no subject) Third Edition
7.747862100601196
PREDICTION >>  Course Materials E234 Course Materials E234 TRCO-1 PART 1 E234 ENRON ONLINE RE: ENHANCED LIMIT ORDER OPTION ENRON ONLINE RE: ENHANCED LIMIT ORDER OPTION Second Law Contents
7.749505519866943
PREDICTION >>  E234 Course Materials Second Law Second LawChallenges Second Law Review [Fwd: Quantum Limits to the Second Law] Book 2 Energy Conservation
7.75689172744751
PREDICTION >>  First Law of Energy Second Law of Energyheat engines Second Law equilibrium statements Second Law Second Law Article on the Second Law
7.752586126327515
PREDICTION >>  Second Law; Entropy and the Second Law Second Law Second Law to read or not read, that is the question In 888-320-4449 Entropy and other formalisms Grupo Calidra Lecture Notes on Entropy and the Second Law
7.747293949127197
PREDICTION >>  Second Law gener

InvalidArgumentError: ignored

In [ ]:
infer = imported.signatures["serving_default"]
print(infer.inputs)

In [ ]:
output = infer(tf.constant([example.SerializeToString()]))
text_eval.ids2str(encoder, output['outputs'].numpy(), None)

In [ ]:
input_serial = tf.concat([tf.constant([example.SerializeToString()]),tf.constant([example.SerializeToString()])],axis=0)

In [ ]:
input_serial

In [ ]:
%load_ext tensorboard

In [ ]:
dir = model_dir[MODELNAME]
!python import_pb_to_tensorboard.py --model_dir $dir --log_dir /tmp/tensorflow_logdir

In [ ]:
%tensorboard --logdir=/tmp/tensorflow_logdir

In [ ]:
import text_eval
import public_parsing_ops
import tensorflow as tf
import numpy as np

_SPM_VOCAB = 'ckpt/c4.unigram.newline.10pct.96000.model'
encoder = public_parsing_ops.create_text_encoder("sentencepiece",
                                                 _SPM_VOCAB)
shapes = {
    'cnn_dailymail': (1024, 128),
    'gigaword':(128, 32)
}

model_dir = {
    'cnn_dailymail': 'model/1595037231/',
    'gigaword': 'model/1595095633/'
}

imported = tf.saved_model.load(model_dir[MODELNAME], tags='serve')
example = tf.train.Example()

text = open(TEXT, "r", encoding="utf-8").read()


shape,_ = shapes[MODELNAME]

input_ids = encoder.encode(text)


inputs = np.zeros(shape)
idx = len(input_ids)
if idx>shape: idx =shape

inputs[:idx] = input_ids[:idx]

example.features.feature["inputs"].int64_list.value.extend(inputs.astype(int))
output = imported.signatures['serving_default'](examples=tf.constant([example.SerializeToString()]))

print("\nPREDICTION >> ", text_eval.ids2str(encoder, output['outputs'].numpy(), None))